In [141]:
#output:8.53585

import pandas as pd
import numpy as np
import csv
import math


In [142]:
train = pd.read_csv('data/train.csv',encoding = "big5")
topic = ['id','測項']+[i for i in range(9)]
for i in train:
    train[i].replace({'NR':0.0},inplace = True)
#雨水nr換成0



#不需要所有的試驗考慮因子,有鑑於相關係數判定目前有：
#CO(2，後五個較高有>0.3)、NMHC(3)(0.14-0.25)、NO2(5)(0.2-0.3)、NOx(6)(0.17-0.28)、PM10(8)、過往PM2.5(9)
#第10小時的PM2.5值為answer。一個月有480小時，總共可作480-9=473次
#為了增加training，把資料從每天換成每個月，為此，要將天數往旁移

#train model目前採用參數：CO,PM10,PM2.5,取前七個小時數值


n_month = 12
feature_count = 3
n_day = 20
new_train = pd.DataFrame()


CO = train[train['測項']=='CO'].iloc[:,3:]
PM2_5 = train[train['測項']=='PM2.5'].iloc[:,3:]
PM10 = train[train['測項']=='PM10'].iloc[:,3:]
train = pd.DataFrame(pd.concat([CO,PM10,PM2_5]).sort_index().values).apply(pd.to_numeric, errors='ignore')

for i in range(n_month):
    month = pd.DataFrame(train.iloc[i*feature_count*n_day:i*feature_count*n_day+feature_count,:].values)
    for j in range(n_day-1):
        day = pd.DataFrame(train.iloc[i*feature_count*n_day+(j+1)*feature_count:i*feature_count*n_day+(j+2)*feature_count,:].values)
        day.columns = [24*(j+1)+i for i in range(24)]
        month = month.join(day)
    new_train=new_train.append(month)
train = pd.DataFrame(new_train.values)

In [143]:
x_train = []
y_train=[]


for j in range(0,train.shape[0],feature_count):
    for i in range(train.shape[1]-9):
        x = train.iloc[j:j+feature_count,i+2:i+9].values
        x_train.append(x.flatten())
        y = train.iloc[j+feature_count-1,i+9]
        y_train.append(y)
X = np.array(x_train,float)
Y = np.array(y_train,float)
#feature scaling
minx = np.min(X, axis=0)
maxx = np.max(X, axis=0)
X=(X - minx) / (maxx - minx)
X = np.insert(X,0,1,axis = 1)

In [218]:
b=0
w  =  np.zeros(len(X[0]))
s_g  = np.zeros(len(X[0]))
lr = 100
lamba = 0.01
landa  =  10^-7
x_t  =  X.transpose()
run = 5000
l2Norm = 0
RMSELoss=[]

for i in range(run):
    tem_y = np.dot(X,w)
    loss = tem_y-Y
    theta = w.copy()
    theta[0] = 0 
    gradient = 2 * np.dot(x_t,loss) + lamba*(theta**2)
    s_g += gradient ** 2
    adagrad = np.sqrt(s_g)
    w = w - lr * gradient / (adagrad+landa)
    RMSELoss.append(np.sqrt(np.mean(loss**2)))
    l2_norm = lamba/ (2*len(X[0]))*sum(w**2)
#    print(theta)

In [197]:
topic = ['id','測項']+[i for i in range(9)]
test = pd.read_csv('data/test.csv',names = topic)
for i in test:
    test[i].replace({'NR':0.0},inplace = True)

#test = test.iloc[:,2:].apply(pd.to_numeric, errors='ignore')
PM2_5_t = test[test['測項']=='PM2.5'].iloc[:,2:]
CO_t = test[test['測項']=='CO'].iloc[:,2:]
PM10_t = test[test['測項']=='PM10'].iloc[:,2:]
test = pd.DataFrame(pd.concat([CO_t,PM10_t,PM2_5_t]).sort_index().values).apply(pd.to_numeric, errors='ignore')

x_test = []
for j in range(0,test.shape[0],feature_count):
    count = test.shape[1]-9
    if count > 0:
        for i in range(test.shape[1]-9):
            x_t = np.array(test.iloc[j:j+feature_count,i+2:i+9])
            x_test.append(x_t.flatten())
    else:
        x_t = np.array(test.iloc[j:j+feature_count,2:])
        x_test.append(x_t.flatten())
X_t = np.array(x_test,float)
minx_t = np.min(X_t, axis=0)
maxx_t = np.max(X_t, axis=0)
X_t=(X_t - minx) / (maxx - minx)
X_test = np.insert(X_t,0,1,axis = 1)
Y_test = np.dot(X_test,w)
id_ = np.array([])
for i in range(len(Y_test)):
    id_=np.append(id_,'id_'+str(i))
final = pd.DataFrame(np.concatenate(([id_],[Y_test])).transpose(),columns = ['id','value'])
final.to_csv('sampleSubmission.csv',index = False)